# README
This cookbook was created by Joe Constantino on June 25, 2024. Shout out to Patrick Green for doing the initial work to prove how you can query Headless BI from a notebook.

This cookbook can be configured to query a Tableau Published data source in natural language using the Headless BI REST endpoint. It uses langchain libraries and the langchain development framework to generate a headless BI request payload from a natural language input query. It then executes a chain to generate an answer to the user's input query.

More on the motivation for this prototype can be found in:


*   [Demo](https://drive.google.com/drive/u/0/folders/1drsQBnkA2ZQAgP2Okwo19t9Xdl-wuisM) and [explainer deck](https://docs.google.com/presentation/d/1_MoYuf28cKNtASp10aW3RJbBcIXSl8vnnijQXqzWWjI/edit#slide=id.g2e7ddaa96e5_0_220)
*   PRD (still a work in progress)
[link text](https://docs.google.com/document/d/1S3gWP4b_ZzF_a4uMcG-zZ1Uu7B3mph78VyvAza5hxyQ/edit#heading=h.dp8s1hcyd5za)


To get started with this notebook, make a copy and add your keys to the vault. More info on headless BI configuration requirements can be found [here](https://docs.google.com/document/d/1_um4JzBCYYtzAECduSTKmeY-COQSvAZo5ISyobR4LPY/edit) and [here](https://docs.google.com/document/d/1C5uCVhelRBKO9L40Asm1Cr3EbXBkLPINbxEYdVEvhII/edit#heading=h.fsq4357uza2).

In [1]:
# %pip install --upgrade --quiet langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.3 MB/s eta 0:00:00


In [ ]:
# %pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.2 MB/s eta 0:00:00


In [ ]:
# %pip install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.5/202.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00


In [1]:
#import all required libraries
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import pandas as pd
import os
import json
import requests
import re

from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

# loading environment variables
load_dotenv()
env_vars = os.environ

# you only need to mount your drive if you have stored the headless BI system prompt in your drive
#drive.mount('/content/drive')




In [2]:
#read in the datasource metadata of the datasource that is listed in vault
def read():
    url = env_vars['READ_METADATA']
    payload = json.dumps({
        "connection": {
            "tableauServerName": env_vars['TABLEAU_DOMAIN'],
            "siteId": env_vars['SITE_NAME'],
            "datasource": env_vars['DATA_SOURCE']
        },
    })

    headers = {
    'Credential-Key': env_vars['PAT_NAME'],
    'Credential-value': env_vars['PAT_SECRET'],
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response data
        #print("Response Data:")
        #print(response.json())
        data = response.json()['data']
        # Create a pandas DataFrame from the JSON data
        #df = pd.DataFrame(data)

        # Display the first few rows of the DataFrame
        #print("Table view of data from the public REST API:")
        #print(df.head())
        #print(df.all())
        return data
        #display(df.head())
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        print(response.text)
datasource_metadata = read()

In [15]:
#define the headless BI query template
def send(query):
    url = env_vars['VDS_URL']
    payload = json.dumps({
        "connection": {
            "tableauServerName": env_vars['TABLEAU_DOMAIN'],
            "siteId": env_vars['SITE_NAME'],
            "datasource": env_vars['DATA_SOURCE']
        },
        "query": query
    })

    headers = {
    'Credential-Key': env_vars['PAT_NAME'],
    'Credential-value': env_vars['PAT_SECRET'],
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response data
        #print("Response Data:")
        #print(response.json())
        data = response.json()['data']
        # Create a pandas DataFrame from the JSON data
        df = pd.DataFrame(data)

        # Display the first few rows of the DataFrame
        #print("Table view of data from the public REST API:")
        #print(df.head())
        #print(df.all())
        return df
        #display(df.head())
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        print(response.text)

In [17]:
#load and instantiate prompt; link to base system prompt: https://drive.google.com/file/d/1wqmTSKIwDPgpucwLAgalEyumO2egDJJ5/view?usp=drive_link

# Specify the path to the system prompt
path = './prompts/nlq_to_vds.py'

# read in system prompt
with open(path, 'r') as json_file:
    headless_bi_prompt_string = json_file.read()

json_string = headless_bi_prompt_string.replace('prompt = ', '', 1).strip()

headless_bi_prompt = json.loads(json_string)
# add datasource metadata of the connected datasource to the system prompt
headless_bi_prompt['data_model'] = datasource_metadata

#convert prompt back to a string to pass in LLM request
ready_prompt = json.dumps(headless_bi_prompt)
print(ready_prompt)

{"instructions": "You are an expert at writing JSON payloads for VDS queries. \nVDS queries are passed in as a JSON body that describes the query, and the return is passed back as an array of JSON objects. The VDS query is a JSON object. It contains three fundamental components.\n1. columns [required] - an array of columns that define the desired output of the query\n2. filters [optional] - an array of filters to apply to the query. They can include fields that are not in the columns array.\n3. options [optional] - metadata that can be used to adjust the behavior of the query. As of now, this is just whether the return format is OBJECTS or ARRAYS (objects is human readable, arrays is more compact).\nYour task is to retrieve data relevant to a natural language query. A pandas AI Agent will be used to transform and analyze the data that your VDS query returns. Don't try to do too much with the json query. You will be successful if you bring back all the data that is required to answer th

In [18]:
def get_utterance():
  query = input('what would you like to know about your data? reply stop if you are done.')
  return query


In [19]:
active_utterance = get_utterance()
while (active_utterance != 'stop'):
  llm = ChatOpenAI(model="gpt-4o")

  active_prompt_template = ChatPromptTemplate.from_messages([
      SystemMessage(content=headless_bi_prompt_string),
      ("user", "{utterance}")])
  #get_utterance = input('what would you like to know about your data? ')
  prompt_value = active_prompt_template.invoke(
      {
          "utterance": active_utterance
      }
  )
  output_parser = StrOutputParser()
  chain = active_prompt_template | llm | output_parser
  output = chain.invoke(active_utterance)

  #print reasoning
  print(output.split('JSON_payload')[0])

  #parse LLM output and query headless BI
  parsed_output = output.split('JSON_payload')[1]
  match = re.search(r'{.*}', parsed_output, re.DOTALL)
  if match:
    json_string = match.group(0)
    payload = json.loads(json_string)
  #print(payload)
  df = send(payload)
  agent = create_pandas_dataframe_agent(
      ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125"),
      df,
      verbose=True,
      agent_type=AgentType.OPENAI_FUNCTIONS,
      allow_dangerous_code=True
  )
  instruct_header = "answer the following query directly by executing the necessary python code. Give a succinct explanation of the steps you took and how you know the answer is correct: "
  agent.invoke(instruct_header + active_utterance)
  active_utterance = get_utterance()

what would you like to know about your data? reply stop if you are done. what are sales by region?


Reasoning: To retrieve sales data by region, we need to query the `Region` and `Sales` columns. We'll use the SUM function to aggregate the sales data for each region.




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "sales_by_region = df.groupby('Region')['SUM(Sales)'].sum()\nprint(sales_by_region)"}`


Region
Central    503170.67
East       691828.17
South      391721.91
West       739813.61
Name: SUM(Sales), dtype: float64
The Python code grouped the dataframe `df` by the 'Region' column and calculated the sum of sales for each region. The result shows the total sales for each region:
- Central: $503,170.67
- East: $691,828.17
- South: $391,721.91
- West: $739,813.61

This information provides the sales figures for each region in the dataset.

> Finished chain.


what would you like to know about your data? reply stop if you are done. what were the unprofitable orders in the corporate segment? 


Reasoning:
To identify unprofitable orders in the corporate segment, we need to retrieve order details where the profit is less than or equal to zero and the segment is "Corporate". The relevant columns would include "Order ID", "Segment", and "Profit".




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "unprofitable_orders = df[(df['Segment'] == 'Corporate') & (df['Profit'] < 0)]\nunprofitable_orders"}`


           Order ID    Segment    Profit
0    US-2024-117723  Corporate -337.8060
1    US-2024-103765  Corporate  -24.7716
2    US-2022-117184  Corporate  -49.9200
3    US-2024-113481  Corporate  -12.5280
4    US-2023-152051  Corporate  -50.8704
..              ...        ...       ...
574  US-2024-158246  Corporate   -2.6997
575  US-2024-169404  Corporate  -29.0536
576  US-2023-144057  Corporate  -27.7920
577  US-2022-125185  Corporate   -7.0980
578  US-2022-131359  Corporate  -45.9954

[562 rows x 3 columns]The unprofitable orders in the corporat

what would you like to know about your data? reply stop if you are done. show me the top 10 most unprofitable orders in the corporate segment and which products were in the orders?


Reasoning: To answer the query, we need to retrieve the top 10 most unprofitable orders in the corporate segment and the products included in those orders. We will include columns for the order ID, product name, segment, and profit. We'll filter by the corporate segment and sort the results to get the bottom 10 orders by profit.




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "unprofitable_orders = df[df['Segment'] == 'Corporate'].sort_values('Profit').head(10)\nunprofitable_orders[['Order ID', 'Product Name', 'Profit']]"}`


            Order ID                                       Product Name  \
804   US-2024-168116          Cubify CubeX 3D Printer Triple Head Print   
2485  US-2024-122714               Ibico EPK-21 Electric Binding System   
670   US-2024-148551   GBC Ibimaster 500 Manual ProClick Binding System   
1489  US-2022-105571         GBC DocuBind TL300 Electric Binding System   
2204  US-2024-159100                       Balt Solid 

what would you like to know about your data? reply stop if you are done. stop
